In [1]:
import pandas as pd
import numpy as np
import os
import deepchem as dc
import pubchempy as pcp
import matplotlib.pyplot as plt
import time
from os.path import dirname, join as pjoin
from deepchem.utils.typing import RDKitMol
from deepchem.feat.base_classes import MolecularFeaturizer
from rdkit import Chem
from rdkit.Chem import AllChem
from numpy.random import random_sample as rs, randint as r
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.model_selection import cross_val_predict as CV
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import datasets, model_selection
from sklearn import svm, model_selection
from sklearn import metrics

C:\tools\Anaconda3\envs\softdrugVirtEnv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\tools\Anaconda3\envs\softdrugVirtEnv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\tools\Anaconda3\envs\softdrugVirtEnv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\tools\Anaconda3\envs\softdrugVirtEnv\

In [2]:
def load_init_data(csv_file='./data.csv'):
    loaded_data = pd.read_csv(csv_file)
    n_files = len(loaded_data)
    Ids = loaded_data["SMILES"]
    labels = loaded_data["Affinity"]
    labels = labels.to_numpy()
    return Ids, labels

In [3]:
def load_round_data(csv_file='./data.csv'):
    loaded_data = pd.read_csv(csv_file)
    n_files = len(loaded_data)
    Ids = loaded_data["Ids"]
    labels = loaded_data["Labels"]
    labels = labels.to_numpy()
    return Ids, labels

In [4]:
def load_featured_data(csv_file):
    loaded_data = pd.read_csv(csv_file)
    samples , feat_count = loaded_data.shape
    label_idx = '%s' %(feat_count - 2)
    features = np.zeros((samples , (feat_count - 2)))
    n_files = len(loaded_data)
    print(loaded_data.shape)
    Ids = loaded_data['0']
    for feat in range(1,(feat_count - 2)):
        features[:, feat] = loaded_data['%s' %feat]
    labels = loaded_data[label_idx]
    labels = labels.to_numpy()
    return Ids, features, labels

In [5]:
def split_data(Ids, Labels):
    X = Ids
    y = Labels
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.3)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5)
    train_data = {'Ids':X_train, 'Labels': y_train}
    validation_data = {'Ids':X_val, 'Labels': y_val}
    test_data = {'Ids':X_test, 'Labels': y_test}
    train_data = pd.DataFrame(data=train_data)
    validation_data = pd.DataFrame(data=validation_data)
    test_data = pd.DataFrame(data=test_data)
    train_data.to_csv('CATS_train_dataset.csv')
    validation_data.to_csv('CATS_validation_dataset.csv')
    test_data.to_csv('CATS_test_dataset.csv')

In [6]:
def featurizing_MACKeys(Id_array, label_array):
    n_samples = len(Id_array)
    features_and_labels = np.zeros((n_samples,169), dtype = int)
    features_and_labels = features_and_labels.astype(str)
    labels = np.zeros(n_samples)
    MACKEYS_featurizer = dc.feat.MACCSKeysFingerprint()
    for idx in range(n_samples):
        mol = Id_array[idx]
        try:
            features_and_labels[idx,1:168] = MACKEYS_featurizer.featurize(([Chem.MolFromSmiles(mol)]))
            features_and_labels[idx, -1] = label_array[idx]
            features_and_labels[idx,0] = mol
        except:
            print('%s Featurization Failed' % (mol))
    features_and_labels = features_and_labels[~np.all(features_and_labels == '0', axis=1)]
    features_and_labels = pd.DataFrame(data=features_and_labels)
    features_and_labels.to_csv('CATS_MACKeys_train_data.csv')

In [27]:
def featurizing_CF(Id_array, label_array):
    n_samples = len(Id_array)
    features_and_labels = np.zeros((n_samples,2050), dtype = int)
    features_and_labels = features_and_labels.astype(str)
    labels = np.zeros(n_samples)
    CF_featurizer = dc.feat.CircularFingerprint()
    for idx in range(n_samples):
        mol = Id_array[idx]
        try:
            features_and_labels[idx,1:2049] = CF_featurizer.featurize(([Chem.MolFromSmiles(mol)]))
            features_and_labels[idx, -1] = label_array[idx]
            features_and_labels[idx,0] = mol
        except:
            print('%s Featurization Failed' % (mol))
    features_and_labels = features_and_labels[~np.all(features_and_labels == '0', axis=1)]
    features_and_labels = pd.DataFrame(data=features_and_labels)
    features_and_labels.to_csv('CATS_CircularFP_test_data.csv')

In [8]:
def featurizing_PubFP(Id_array, label_array):
    n_samples = len(Id_array)
    features_and_labels = np.zeros((n_samples,883), dtype = int)
    features_and_labels = features_and_labels.astype(str)
    labels = np.zeros(n_samples)
    PubFB_featurizer = dc.feat.PubChemFingerprint()
    for idx in range(n_samples):
        mol = Id_array[idx]
        try:
            features_and_labels[idx,1:882] = PubFB_featurizer.featurize(([Chem.MolFromSmiles(mol)]))
            features_and_labels[idx, -1] = label_array[idx]
            features_and_labels[idx,0] = mol
        except:
            print('%s Featurization Failed' % (mol))
    features_and_labels = features_and_labels[~np.all(features_and_labels == '0', axis=1)]
    features_and_labels = pd.DataFrame(data=features_and_labels)
    features_and_labels.to_csv('CATS_PubChemFP_train_data.csv')

In [9]:
def featurizing_RDKit(Id_array, label_array):
    n_samples = len(Id_array)
    features_and_labels = np.zeros((n_samples,202), dtype = int)
    features_and_labels = features_and_labels.astype(str)
    labels = np.zeros(n_samples)
    RDKit_featurizer = dc.feat.RDKitDescriptors()
    for idx in range(n_samples):
        mol = Id_array[idx]
        try:
            features_and_labels[idx,1:201] = RDKit_featurizer.featurize(([Chem.MolFromSmiles(mol)]))
            features_and_labels[idx, -1] = label_array[idx]
            features_and_labels[idx,0] = mol
        except:
            print('%s Featurization Failed' % (mol))
    features_and_labels = features_and_labels[~np.all(features_and_labels == '0', axis=1)]
    features_and_labels = pd.DataFrame(data=features_and_labels)
    features_and_labels.to_csv('CATS_RDKit_train_data.csv')

In [10]:
def KnnLooCV(features, labels, k):
    X = features
    y = labels
    model = KNR(n_neighbors = k)
    pred = CV(model, X, y, cv=len(X))
    r2_score = r2(pred,y)
    print(r2_score)
    return r2_score

In [12]:
def feature_importance(feature_arrays, labels):
    y = labels
    x = feature_arrays
    _, n_features = feature_arrays.shape
    importance_values = np.zeros(n_features)
    x_train, x_test, y_train, y_test = model_selection.train_test_split(
        x, y, test_size = 0.2, random_state = 1)
    kf = model_selection.KFold(n_splits = 5)
    for train_idx, val_idx in kf.split(x_train):
        feature_imp_model = RandomForestRegressor(
            n_estimators = 1000, criterion="mse",
            min_samples_leaf = 1, max_features = "sqrt")
        feature_imp_model.fit(x_train[train_idx], y_train[train_idx])
        fold_feat_imp = feature_imp_model.feature_importances_
        importance_values += fold_feat_imp
    print(importance_values)
    return importance_values

In [28]:
def feature_rearrage(feature_arrays, importance_values):
    samples, n_features = feature_arrays.shape
    ranked_feature_array = np.zeros((samples, n_features))
    new_order = np.argsort(importance_values)
    count = n_features - 1
    for new_ind in new_order:
        ranked_feature_array[:,count] = feature_arrays[:,new_ind]
        count -= 1
    ranked_features = pd.DataFrame(data=ranked_feature_array)
    ranked_features.to_csv('ranked_featured_CFP_test_data.csv')
    return ranked_feature_array

In [20]:
def SVM_optimization(train_feats, train_labels, val_feats, val_labels):
    train_features = train_feats
    y_train = train_labels
    val_features = val_feats
    y_val = val_labels
    _, total_features = train_features.shape
    kernel = 'rbf'
    lmda = 0.0010
    MSEs = np.zeros(total_features)
    R2s = np.zeros(total_features)
    times = np.zeros(total_features)
    for feats_used in range(1,total_features):
        x_train = train_features[: , 0:feats_used]
        x_val = val_features[: , 0:feats_used]
        prev_time = time.time()
        svc = svm.SVR(kernel=kernel, C=1/lmda)
        svc.fit(x_train, y_train)
        curr_time = time.time()
        y_pred = svc.predict(x_val)
        MSEs[feats_used] = metrics.mean_squared_error(y_val, y_pred)
        R2s[feats_used] = metrics.r2_score(y_val, y_pred)
        times[feats_used] = curr_time - prev_time
    return MSEs, R2s, times

In [25]:
def final_model(train_feats, train_labels, fin_feats, fin_labels):
    train_features = train_feats
    y_train = train_labels
    test_features = fin_feats
    y_test = fin_labels
    _, total_features = train_features.shape
    kernel = 'rbf'
    lmda = 0.0010
    x_train = train_features[: , 0:38]
    x_test = test_features[: , 0:38]
    prev_time = time.time()
    svc = svm.SVR(kernel=kernel, C=1/lmda)
    svc.fit(x_train, y_train)
    curr_time = time.time()
    y_pred = svc.predict(x_test)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    R2 = metrics.r2_score(y_test, y_pred)
    timed = curr_time - prev_time
    return MSE, R2, timed

In [11]:
Ids_tot, labels_tot = load_init_data('CATS Id and Label.csv')
split_data(Ids_tot, labels_tot)
Ids_train, labels_train = load_round_data('CATS_train_dataset.csv')
featurizing_MACKeys(Ids_train, labels_train)
featurizing_CF(Ids_train, labels_train)
featurizing_PubFP(Ids_train, labels_train)
featurizing_RDKit(Ids_train, labels_train)
Ids_Mac, features_Mac, labels_Mac = load_featured_data('CATS_MACKeys_train_data.csv')
Ids_CFP, features_CFP, labels_CFP = load_featured_data('CATS_CircularFP_train_data.csv')
Ids_PCFP, features_PCFP, labels_PCFP = load_featured_data('CATS_PubChemFP_train_data.csv')
Ids_RD, features_RD, labels_RD = load_featured_data('CATS_RDKit_train_data.csv')
features_RD_noNaN = np.nan_to_num(features_RD, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
r2_score_Mac = KnnLooCV(features_Mac, labels_Mac, 1)
r2_score_CFP = KnnLooCV(features_CFP, labels_CFP, 1)
r2_score_RD = KnnLooCV(features_RD_noNaN, labels_RD, 1)
r2_score_PCFP = KnnLooCV(features_PCFP, labels_PCFP, 1)
r2_score_Mac = KnnLooCV(features_Mac, labels_Mac, 2)
r2_score_CFP = KnnLooCV(features_CFP, labels_CFP, 2)
r2_score_RD = KnnLooCV(features_RD_noNaN, labels_RD, 2)
r2_score_PCFP = KnnLooCV(features_PCFP, labels_PCFP, 2)
r2_score_Mac = KnnLooCV(features_Mac, labels_Mac, 3)
r2_score_CFP = KnnLooCV(features_CFP, labels_CFP, 3)
r2_score_RD = KnnLooCV(features_RD_noNaN, labels_RD, 3)
r2_score_PCFP = KnnLooCV(features_PCFP, labels_PCFP, 3)
r2_score_Mac = KnnLooCV(features_Mac, labels_Mac, 5)
r2_score_CFP = KnnLooCV(features_CFP, labels_CFP, 5)
r2_score_RD = KnnLooCV(features_RD_noNaN, labels_RD, 5)
r2_score_PCFP = KnnLooCV(features_PCFP, labels_PCFP, 5)

Failed to featurize datapoint 0, CC(C)NCc1cc(C#Cc2cc(-c3nn(CCCN4CCOCC4C)c4c3CN(C(=O)C(N)=O)CC4)ccc2Cl)ccc1Cl. Appending empty array
Exception message: 'NoneType' object is not subscriptable


C[C@H]1COCCN1CCCn2c3c(c(n2)c4ccc(c(c4)C#Cc5ccc(c(c5)CNC(C)C)Cl)Cl)CN(CC3)C(=O)C(=O)N Featurization Failed


Failed to featurize datapoint 0, CC1CN(CCCn2nc(-c3ccc(Cl)c(C#Cc4ccc(CNCc5ccc(Cl)cc5)cc4)c3)c3c2CCN(S(C)(=O)=O)C3)CC(C)O1. Appending empty array
Exception message: 'NoneType' object is not subscriptable


C[C@@H]1CN(C[C@@H](O1)C)CCCn2c3c(c(n2)c4ccc(c(c4)C#Cc5ccc(cc5)CNCc6ccc(cc6)Cl)Cl)CN(CC3)S(=O)(=O)C Featurization Failed


Failed to featurize datapoint 0, CC1COCCN1CCCn1nc(-c2ccc(Cl)c(C#Cc3ccc(Cl)c(CNCC(C)(C)O)c3)c2)c2c1CCN(C(=O)C(N)=O)C2. Appending empty array
Exception message: 'NoneType' object is not subscriptable


...

c1cc(cc(c1)C#Cc2cc(ccc2Cl)c3c4c(n(n3)CCCN5CCSCC5)CCN(C4)C(=O)N)CNCc6cccc(c6)F Featurization Failed


Failed to featurize datapoint 0, NC(=O)N1CCc2c(c(-c3ccc(Cl)c(C#Cc4ccc(CNCc5ccc(Cl)cc5)cc4)c3)nn2CCCN2CCC3(CC2)OCCO3)C1. Appending empty array
Exception message: 'NoneType' object is not subscriptable


c1cc(ccc1CNCc2ccc(cc2)Cl)C#Cc3cc(ccc3Cl)c4c5c(n(n4)CCCN6CCC7(CC6)OCCO7)CCN(C5)C(=O)N Featurization Failed


Failed to featurize datapoint 0, CC1COCCN1CCCn1nc(-c2ccc(Cl)c(C#Cc3ccc(Cl)c(C(=O)NCCCNC(=O)OC(C)(C)C)c3)c2)c2c1CCN(C(=O)C(N)=O)C2. Appending empty array
Exception message: 'NoneType' object is not subscriptable


C[C@H]1COCCN1CCCn2c3c(c(n2)c4ccc(c(c4)C#Cc5ccc(c(c5)C(=O)NCCCNC(=O)OC(C)(C)C)Cl)Cl)CN(CC3)C(=O)C(=O)N Featurization Failed
(321, 170)
(321, 2051)
(307, 884)
(321, 203)
-0.6371614623809048
-0.6153391727676052
-1.1251955987526756
-0.7646398356729585
-2.179820578349534
-1.5200891623719097
-2.6201066030103926
-1.468900539323681
-3.5489059091778286
-2.3325686309436606
-3.356666904836853
-2.310795211867081
-5.960131681573764
-2.7026764648895374
-4.269397755834009
-4.638163605327216


In [13]:
importance_values = feature_importance(features_CFP, labels_CFP)

[0.         0.         0.00926409 ... 0.00059225 0.00058392 0.        ]


In [ ]:
for item in importance_values:
    print(item)

In [16]:
ranked_train_feature_array = feature_rearrage(features_CFP, importance_values)

In [19]:
Ids_val, labels_val = load_round_data('CATS_validation_dataset.csv')
featurizing_CF(Ids_val, labels_val)
Ids_CFP_val, features_CFP_val, labels_CFP_val = load_featured_data('CATS_CircularFP_val_data.csv')
ranked_val_feature_array = feature_rearrage(features_CFP_val, importance_values)

(69, 2051)


In [21]:
MSEs, R2s, times = SVM_optimization(ranked_train_feature_array, labels_CFP, ranked_val_feature_array, labels_CFP_val)

In [29]:
Ids_test, labels_test = load_round_data('CATS_test_dataset.csv')
featurizing_CF(Ids_test, labels_test)
Ids_CFP_test, features_CFP_test, labels_CFP_test = load_featured_data('CATS_CircularFP_test_data.csv')
ranked_test_feature_array = feature_rearrage(features_CFP_test, importance_values)
MSE, R2, time = final_model(ranked_train_feature_array, labels_CFP, ranked_test_feature_array, labels_CFP_test)

(69, 2051)


In [30]:
print(MSE, R2, time)

0.39699614756317325 -0.11526081519279652 0.0040166378021240234
